In [1]:
#Importing numeric table
import pandas as pd
training_df = pd.read_csv('/home/data/MSA8010/msa8010f16t03/data/pr_training_7l_p5.csv')

In [2]:
# dropping the first column from categorical_with_id_df table
training_df2=training_df.drop(training_df.columns[[0]], axis=1)

In [3]:
#training_df2.head(5)
len(training_df2)

710248

In [4]:
validation_df = pd.read_csv('/home/data/MSA8010/msa8010f16t03/data/pr_test_5l_p5.csv')

In [5]:
# dropping the first column from categorical_with_id_df table
validation_df2=validation_df.drop(validation_df.columns[[0]], axis=1)

In [6]:
#training_df2.head(5)
len(validation_df2)

473499

In [7]:
import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import cross_validation, metrics   #Additional scklearn functions
from sklearn.grid_search import GridSearchCV   #Perforing grid search

import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 4

train = training_df2
target = 'Response'
IDcol = 'Id'
validation=validation_df2

/usr/lib64/python3.4/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/lib64/python3.4/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


4127

In [8]:
dvalidation_predictions=pd.DataFrame()

def modelfit(alg, dtrain, dvalidation,predictors,useTrainCV=True, cv_folds=5, early_stopping_rounds=10):
    
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(dtrain[predictors].values, label=dtrain[target].values)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
            metrics='auc', early_stopping_rounds=early_stopping_rounds,callbacks=[xgb.callback.print_evaluation(show_stdv=True)])
        alg.set_params(n_estimators=cvresult.shape[0])
        print("Number of boosted trees is n_estimaors which is %d" %(cvresult.shape[0]))
        print(cvresult)
    
    #Fit the algorithm on the data
    alg.fit(dtrain[predictors], dtrain['Response'],eval_metric='auc')
        
    #Predict training set:
    dtrain_predictions = alg.predict(dtrain[predictors])
    dtrain_predprob = alg.predict_proba(dtrain[predictors])[:,1]
    
    #Predict validation set:
    dvalidation_predictions = alg.predict(dvalidation[predictors])
   
    dvalidation_predprob = alg.predict_proba(dvalidation[predictors])[:,1]
    
    
        
#Print model report:
    print("\nTotal Number of 1's predicted in the training dataset is %d" %(sum(dtrain_predictions)))  
    print("\nActual Number of 1's in the training dataset is %d" %(sum(dtrain.Response)) )
    print("\nTotal number of 0's predicted in the training dataset is %d" %(len(dtrain_predictions)-sum(dtrain_predictions)))
    print("\nTotal number of 0's in the training dataset is %d" %(len(dtrain)-sum(dtrain.Response)))
    print ("\nModel Report")
    print ("Accuracy : %.4g" % metrics.accuracy_score(dtrain['Response'].values, dtrain_predictions))
    print ("AUC Score (Train): %f" % metrics.roc_auc_score(dtrain['Response'], dtrain_predprob))
                    
    #feat_imp = pd.Series(alg.booster().get_fscore()).sort_values(ascending=False)
    #print(feat_imp)
    #feat_imp.plot(kind='bar', figsize=(15,10),title='Feature Importances')
    #plt.ylabel('Feature Importance Score')

#Print validation model report:
    print("\nTotal Number of 1's predicted in the validation dataset is %d" %(sum(dvalidation_predictions)))  
    print("\nActual Number of 1's in the validation dataset is %d" %(sum(dvalidation.Response)) )
    print("\nTotal number of 0's predicted in the validation dataset is %d" %(len(dvalidation_predictions)-sum(dvalidation_predictions)))
    print("\nTotal number of 0's in the validation dataset is %d" %(len(dvalidation)-sum(dvalidation.Response)))
    print("\nModel Report on validation set")
    print("Accuracy : %.4g" % metrics.accuracy_score(dvalidation['Response'].values, dvalidation_predictions))
    print("AUC Score (Validation): %f" % metrics.roc_auc_score(dvalidation['Response'], dvalidation_predprob))
    
    return dvalidation_predictions
    

In [9]:
#Choose all predictors except target & IDcols
predictors = [x for x in train.columns if x not in [target, IDcol]]
xgb1 =XGBClassifier(learning_rate =0.9,
 n_estimators=20,
 max_depth=2,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27)
f=modelfit(xgb1, train, validation,predictors)

[0]	train-auc:0.588248+0.0105358	test-auc:0.587805+0.0111015
[1]	train-auc:0.601953+0.00546296	test-auc:0.597316+0.0103689
[2]	train-auc:0.609822+0.00565102	test-auc:0.604246+0.0120949
[3]	train-auc:0.616807+0.00378558	test-auc:0.609905+0.0119735
[4]	train-auc:0.624286+0.00281218	test-auc:0.618762+0.00798054
[5]	train-auc:0.655652+0.00905892	test-auc:0.649728+0.0142096
[6]	train-auc:0.657731+0.00468231	test-auc:0.65054+0.00461305
[7]	train-auc:0.663542+0.00513045	test-auc:0.65581+0.00652974
[8]	train-auc:0.666454+0.00644184	test-auc:0.65557+0.00732638
[9]	train-auc:0.668379+0.00652626	test-auc:0.657701+0.00804422
[10]	train-auc:0.67132+0.00778029	test-auc:0.659485+0.0085315
[11]	train-auc:0.675439+0.00557135	test-auc:0.662743+0.00615078
[12]	train-auc:0.678408+0.00494547	test-auc:0.662272+0.00623656
[13]	train-auc:0.682939+0.00497887	test-auc:0.662294+0.00678103
[14]	train-auc:0.685239+0.00574824	test-auc:0.661681+0.00584852
[15]	train-auc:0.68822+0.00556545	test-auc:0.662098+0.0045176

In [56]:
#creating a confusion matrix from xgboost model
from sklearn.metrics import confusion_matrix
predict=f
actual=validation.Response.as_matrix()

In [57]:
#confusion matrix
confusion_matrix(actual,predict)

array([[470746,      1],
       [  2752,      0]])